In [1]:
import sys

from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader

from hyperopt import fmin, tpe, hp, Trials
from hyperopt.pyll import scope

from simple_cnn import SimpleCnn
sys.path.append('../../utils')
from trainer import Trainer

In [2]:
# Load model.
model = SimpleCnn()

In [3]:
train_dataset = datasets.FashionMNIST(root='../../data/fashion_mnist',
                                train=True,
                                transform=transforms.ToTensor(),
                                download=True)

train_frac = 0.8

train_size = int(train_frac * len(train_dataset))
valid_size = len(train_dataset) - train_size

train_dataset, valid_dataset = random_split(
        train_dataset, [train_size, valid_size])


In [4]:
def objective_function(params):
    config = {
        'optimizer': {
            'name' : params['optimizer'],
            'params' : {
                'lr' : params['lr'],
            }
        },
        'loss_fn' : 'ce_loss',
        'batch_size' : 128,
        'epochs' : 5,
        'device' : 'cpu'
    }
    trainer = Trainer(model, train_dataset, config)
    result = trainer.train()
    return result['loss_history'][-1]

In [5]:
search_space = {
    'lr': hp.uniform('lr', 0.001, 0.1),  # Learning rate
    'optimizer': hp.choice('optimizer', ['sgd', 'adam']),  # Optimizer type
}

In [6]:
trials = Trials()

best_params = fmin(
    fn = objective_function,
    space = search_space,
    algo = tpe.suggest,
    max_evals = 1,
    trials = trials
)

  0%|          | 0/1 [00:00<?, ?trial/s, best loss=?]

100%|##########| 375/375 [00:09<00:00, 38.05it/s]


Epoch 1/5,                   Loss: 0.6382884060541789
  0%|          | 0/1 [00:09<?, ?trial/s, best loss=?]

100%|##########| 375/375 [00:09<00:00, 37.69it/s]


Epoch 2/5,                   Loss: 0.44184005284309386
  0%|          | 0/1 [00:19<?, ?trial/s, best loss=?]

...

Epoch 3/5,                   Loss: 0.41620501899719237
  0%|          | 0/1 [00:30<?, ?trial/s, best loss=?]

100%|##########| 375/375 [00:10<00:00, 36.67it/s]


Epoch 4/5,                   Loss: 0.39439649625619255
  0%|          | 0/1 [00:40<?, ?trial/s, best loss=?]

100%|##########| 375/375 [00:10<00:00, 36.92it/s]


Epoch 5/5,                   Loss: 0.39210060584545137
100%|██████████| 1/1 [00:50<00:00, 50.41s/trial, best loss: 147.03772719204426]


In [7]:
print(best_params)

{'lr': np.float64(0.022792097124131792), 'optimizer': np.int64(1)}
